In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
cars_df = pd.read_csv('cars_processed.csv')

In [ ]:
# 가격 칼럼을 백만 단위로 변환
cars_df['가격'] = cars_df['가격'] / 1_00

# 결과 출력
cars_df

,연식,주행거리,배기량,가격,신차대비가격,소유자변경횟수,보험_내차피해(횟수),보험_내차피해(가격),사고상세_타차가해(횟수),보험_타차피해(가격),...,연료_기타,연료_디젤,보증정보_0,보증정보_1,색상_검정색,색상_기타,색상_흰색,이름_기아,이름_제네시스,이름_현대
0,114,180900,2999,8.20,25,4,3,1981920,0,0,...,0,0,1,0,1,0,0,0,0,1
1,164,265753,2656,3.99,15,0,4,3274363,6,2834910,...,0,0,1,0,1,0,0,0,0,1
2,80,89076,2999,11.20,0,1,2,1904690,0,0,...,0,0,1,0,1,0,0,0,0,1
3,122,111306,2999,7.49,19,5,2,5660104,1,3830176,...,0,0,1,0,1,0,0,0,0,1
4,110,102801,2999,9.90,30,4,7,11149558,3,8119080,...,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,25,38163,2199,39.99,90,1,1,615569,2,3572498,...,0,1,1,0,0,0,1,0,0,1
719,28,13400,3778,41.30,96,0,0,0,1,412580,...,0,1,0,1,0,1,0,0,0,1
720,39,112746,3778,30.80,76,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
721,28,26500,2199,41.30,93,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,1


상관관계 분석

In [ ]:
cars_df.corr()

,연식,주행거리,배기량,가격,신차대비가격,소유자변경횟수,보험_내차피해(횟수),보험_내차피해(가격),사고상세_타차가해(횟수),보험_타차피해(가격),...,연료_기타,연료_디젤,보증정보_0,보증정보_1,색상_검정색,색상_기타,색상_흰색,이름_기아,이름_제네시스,이름_현대
연식,1.000000,0.548829,0.092408,-0.634422,-0.536150,0.431541,0.479514,0.130275,0.324182,0.149924,...,-0.094724,-0.083912,0.636176,-0.636176,0.094280,-0.098977,-0.001835,-0.064284,-0.269604,0.238940
주행거리,0.548829,1.000000,0.103742,-0.435180,-0.338711,0.242958,0.289683,0.006214,0.230411,0.081809,...,-0.073137,-0.022141,0.381118,-0.381118,0.078097,-0.037717,-0.043946,-0.044450,-0.139828,0.134221
배기량,0.092408,0.103742,1.000000,0.350462,-0.115959,0.002485,0.064253,0.046380,-0.098611,-0.073040,...,-0.140734,-0.180437,-0.027118,0.027118,0.334015,0.008671,-0.350852,-0.176972,0.482041,-0.156670
가격,-0.634422,-0.435180,0.350462,1.000000,0.663796,-0.346788,-0.349374,-0.125847,-0.336980,-0.220215,...,0.055092,-0.039548,-0.511157,0.511157,0.090340,0.062328,-0.152377,-0.059555,0.611629,-0.351706
신차대비가격,-0.536150,-0.338711,-0.115959,0.663796,1.000000,-0.282055,-0.252266,-0.104882,-0.258752,-0.175178,...,0.086480,-0.087371,-0.454391,0.454391,-0.041815,0.089597,-0.042980,-0.032200,0.168843,-0.082484
소유자변경횟수,0.431541,0.242958,0.002485,-0.346788,-0.282055,1.000000,0.303821,0.237677,0.233477,0.170748,...,-0.039768,-0.087427,0.388796,-0.388796,0.023381,-0.144514,0.114513,-0.000704,-0.145692,0.097578
보험_내차피해(횟수),0.479514,0.289683,0.064253,-0.349374,-0.252266,0.303821,1.000000,0.500548,0.498333,0.292460,...,0.002761,-0.162385,0.328504,-0.328504,0.150909,-0.106179,-0.053008,-0.042704,-0.146168,0.136820
보험_내차피해(가격),0.130275,0.006214,0.046380,-0.125847,-0.104882,0.237677,0.500548,1.000000,0.265223,0.366499,...,0.059793,-0.103954,0.132694,-0.132694,0.022690,-0.029319,0.004828,-0.063856,-0.013499,0.068163
사고상세_타차가해(횟수),0.324182,0.230411,-0.098611,-0.336980,-0.258752,0.233477,0.498333,0.265223,1.000000,0.611905,...,-0.041323,0.011270,0.264863,-0.264863,0.035566,-0.067576,0.028286,0.039031,-0.185253,0.087070
보험_타차피해(가격),0.149924,0.081809,-0.073040,-0.220215,-0.175178,0.170748,0.292460,0.366499,0.611905,1.000000,...,-0.047218,0.005264,0.196476,-0.196476,0.002281,-0.047907,0.043570,0.050057,-0.140782,0.047265


In [ ]:
corr_data = cars_df.corr()['가격'].abs()
corr_data.sort_values(ascending=False)

,가격
가격,1.000000
신차대비가격,0.663796
연식,0.634422
이름_제네시스,0.611629
보증정보_1,0.511157
보증정보_0,0.511157
주행거리,0.435180
옵션 합,0.432506
이름_현대,0.351706
배기량,0.350462


상관관계가 0.1 수준인 변수 제거

In [ ]:
cars_df.drop(['연료_디젤', '사고 합', '연료_기타', '이름_기아', '색상_기타', '색상_검정색', '연료_가솔린', '보험_내차피해(가격)', '연료_LPG','색상_흰색'], axis=1, inplace=True)
cars_df

,연식,주행거리,배기량,가격,신차대비가격,소유자변경횟수,보험_내차피해(횟수),사고상세_타차가해(횟수),보험_타차피해(가격),옵션 합,보증정보_0,보증정보_1,이름_제네시스,이름_현대
0,114,180900,2999,8.20,25,4,3,0,0,6,1,0,0,1
1,164,265753,2656,3.99,15,0,4,6,2834910,4,1,0,0,1
2,80,89076,2999,11.20,0,1,2,0,0,4,1,0,0,1
3,122,111306,2999,7.49,19,5,2,1,3830176,8,1,0,0,1
4,110,102801,2999,9.90,30,4,7,3,8119080,4,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,25,38163,2199,39.99,90,1,1,2,3572498,2,1,0,0,1
719,28,13400,3778,41.30,96,0,0,1,412580,9,0,1,0,1
720,39,112746,3778,30.80,76,1,0,0,0,9,0,1,0,1
721,28,26500,2199,41.30,93,0,0,0,0,9,1,0,0,1


In [ ]:
corr_data = cars_df.corr()['가격'].abs()
corr_data.sort_values(ascending=False)

,가격
가격,1.000000
신차대비가격,0.663796
연식,0.634422
이름_제네시스,0.611629
보증정보_0,0.511157
보증정보_1,0.511157
주행거리,0.435180
옵션 합,0.432506
이름_현대,0.351706
배기량,0.350462


In [ ]:
fig = px.imshow(cars_df.corr(method='pearson'), # 기본은 피어슨, 상황에 따라 스피어만
                title='Correlations Among Training Features',
                height=700, width=700)
fig.show()

다중공선성 (Multicollinearity) 제거

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF 계산
def calculate_vif(df):
    vif = pd.DataFrame()
    vif["Variable"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif

# 예시 데이터 프레임에서 VIF 계산
vif = calculate_vif(cars_df)
print(vif)

# VIF는 다중공선성이고 see, sst, ssr이 있고 빨간선이 원래 값에 근접하면 sse는 작아지고, ssr이 커진다  빨간선은 variance_inflation_factor를 통해서 알아낸다

         Variable        VIF
0              연식   3.047437
1            주행거리   1.562295
2             배기량   2.073739
3              가격   5.867845
4          신차대비가격   2.807516
5         소유자변경횟수   1.326240
6     보험_내차피해(횟수)   1.691013
7   사고상세_타차가해(횟수)   2.048632
8     보험_타차피해(가격)   1.643783
9            옵션 합   1.515837
10         보증정보_0  23.453273
11         보증정보_1  10.699909
12        이름_제네시스   2.212986
13          이름_현대   1.327323


- 혼돈모델 히든 마르코프 모델 arima lstm 카이제곱 통계량 ARTEMIS Transformer LSTM RandomForset(그리드서치)

In [ ]:
cars_df.drop(['보증정보_1', '배기량'], axis = 1, inplace=True)
vif = calculate_vif(cars_df)
print(vif)

         Variable        VIF
0              연식  12.126336
1            주행거리   5.665245
2              가격  10.147797
3          신차대비가격   4.857079
4         소유자변경횟수   3.398918
5     보험_내차피해(횟수)   3.257975
6   사고상세_타차가해(횟수)   3.176635
7     보험_타차피해(가격)   2.101432
8            옵션 합   7.259422
9          보증정보_0   5.803920
10        이름_제네시스   2.424029
11          이름_현대   2.852127


In [ ]:
fig = px.imshow(cars_df.corr(method='pearson'), # 기본은 피어슨, 상황에 따라 스피어만
                title='Correlations Among Training Features',
                height=700, width=700)
fig.show()

In [ ]:
cars_df.head()

,연식,주행거리,가격,신차대비가격,소유자변경횟수,보험_내차피해(횟수),사고상세_타차가해(횟수),보험_타차피해(가격),옵션 합,보증정보_0,이름_제네시스,이름_현대
0,114,180900,8.20,25,4,3,0,0,6,1,0,1
1,164,265753,3.99,15,0,4,6,2834910,4,1,0,1
2,80,89076,11.20,0,1,2,0,0,4,1,0,1
3,122,111306,7.49,19,5,2,1,3830176,8,1,0,1
4,110,102801,9.90,30,4,7,3,8119080,4,1,0,1


모델링

In [ ]:
X = cars_df.drop('가격', axis=1)
y = cars_df['가격']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
X_train.head()

,연식,주행거리,신차대비가격,소유자변경횟수,보험_내차피해(횟수),사고상세_타차가해(횟수),보험_타차피해(가격),옵션 합,보증정보_0,이름_제네시스,이름_현대
355,150,144601,15,2,3,1,1227660,8,1,0,0
396,62,109682,0,1,0,1,2500000,2,1,0,0
60,79,105750,0,4,1,0,0,7,1,0,1
718,25,38163,90,1,1,2,3572498,2,1,0,1
456,98,193561,41,4,0,0,0,4,1,0,0


In [ ]:
y_train.head()

,가격
355,6.30
396,13.90
60,9.99
718,39.99
456,12.50


In [ ]:
models = [
    ("Linear Regression", LinearRegression()),
    ("Lasso Regression", Lasso(random_state=42)),  # random_state는 Lasso에서 지원됨
    ("Ridge Regression", Ridge(random_state=42)),  # random_state는 Ridge에서 지원됨
    ("Elastic Net Regression", ElasticNet(random_state=42)),  # random_state는 ElasticNet에서 지원됨
    ("Decision Tree Regressor", DecisionTreeRegressor(random_state=42)),
    ("Random Forest Regressor", RandomForestRegressor(random_state=42)),
    ("Gradient Boosting Regressor", GradientBoostingRegressor(random_state=42)),
    ("XGB Regressor", XGBRegressor(random_state=42)),
    ("KNN Regressor", KNeighborsRegressor()),  # KNN Regressor는 random_state를 지원하지 않음
    ("SVR", SVR())  # SVR은 random_state를 지원하지 않음
]

# 모델 평가
for name, model in models:
    model.fit(X_train, y_train)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print(f"{name}: 평균 음의 평균 제곱 오차 (MSE): {-scores.mean()}")

Linear Regression: 평균 음의 평균 제곱 오차 (MSE): 48.72614275565023
Lasso Regression: 평균 음의 평균 제곱 오차 (MSE): 60.38425886059724
Ridge Regression: 평균 음의 평균 제곱 오차 (MSE): 48.712970131858455
Elastic Net Regression: 평균 음의 평균 제곱 오차 (MSE): 72.60031856631035
Decision Tree Regressor: 평균 음의 평균 제곱 오차 (MSE): 58.211488781820606
Random Forest Regressor: 평균 음의 평균 제곱 오차 (MSE): 40.382398955046504
Gradient Boosting Regressor: 평균 음의 평균 제곱 오차 (MSE): 39.11154995913724
XGB Regressor: 평균 음의 평균 제곱 오차 (MSE): 40.674927032700815
KNN Regressor: 평균 음의 평균 제곱 오차 (MSE): 139.55938494260965
SVR: 평균 음의 평균 제곱 오차 (MSE): 186.11187733498087


In [ ]:
from sklearn.metrics import r2_score
models = {
    "Linear Regression": LinearRegression(),
    "Lasso Regression": Lasso(random_state=42),  # random_state는 Lasso에서 지원됨
    "Ridge Regression": Ridge(random_state=42),  # random_state는 Ridge에서 지원됨
    "Elastic Net Regression": ElasticNet(random_state=42),  # random_state는 ElasticNet에서 지원됨
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42),
    "XGB Regressor": XGBRegressor(random_state=42),
    "KNN Regressor": KNeighborsRegressor(),  # KNN Regressor는 random_state를 지원하지 않음
    "SVR": SVR()  # SVR은 random_state를 지원하지 않음
}

# 모델 학습 및 평가
results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    results[model_name] = r2

# 결과 출력
for model_name, r2 in results.items():
    print(f"{model_name}: R² = {r2}")

Linear Regression: R² = 0.8033235766683419
Lasso Regression: R² = 0.8077377953167552
Ridge Regression: R² = 0.8040135656097149
Elastic Net Regression: R² = 0.7716463292085188
Decision Tree Regressor: R² = 0.9162114528909524
Random Forest Regressor: R² = 0.9111223121739317
Gradient Boosting Regressor: R² = 0.9226691829525602
XGB Regressor: R² = 0.9562923647133895
KNN Regressor: R² = 0.3338879316199519
SVR: R² = 0.06453701922133126


In [ ]:
# 모델 정의
model_dt = DecisionTreeRegressor(random_state=42)
model_rf = RandomForestRegressor(random_state=42)
model_gb = GradientBoostingRegressor(random_state=42)
model_xgb = XGBRegressor(random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid_dt = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10],
    'gamma': [0, 0.1, 0.2]
}

# 그리드 서치 정의
grid_search_dt = GridSearchCV(model_dt, param_grid_dt, cv=5, scoring='r2')
grid_search_rf = GridSearchCV(model_rf, param_grid_rf, cv=5, scoring='r2')
grid_search_gb = GridSearchCV(model_gb, param_grid_gb, cv=5, scoring='r2')
grid_search_xgb = GridSearchCV(model_xgb, param_grid_xgb, cv=5, scoring='r2')

# 각 모델에 대해 그리드 서치 수행
grid_search_dt.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)
grid_search_gb.fit(X_train, y_train)
grid_search_xgb.fit(X_train, y_train)

# 최적 하이퍼파라미터로 모델 정의
best_dt = grid_search_dt.best_estimator_
best_rf = grid_search_rf.best_estimator_
best_gb = grid_search_gb.best_estimator_
best_xgb = grid_search_xgb.best_estimator_

# 기본 모델 학습
model_dt.fit(X_train, y_train)
model_rf.fit(X_train, y_train)
model_gb.fit(X_train, y_train)
model_xgb.fit(X_train, y_train)

# 최적 모델 예측
predictions_best_dt = best_dt.predict(X_test)
predictions_best_rf = best_rf.predict(X_test)
predictions_best_gb = best_gb.predict(X_test)
predictions_best_xgb = best_xgb.predict(X_test)

# 기본 모델 예측
predictions_dt = model_dt.predict(X_test)
predictions_rf = model_rf.predict(X_test)
predictions_gb = model_gb.predict(X_test)
predictions_xgb = model_xgb.predict(X_test)

# R² 계산
r2_best_dt = r2_score(y_test, predictions_best_dt)
r2_best_rf = r2_score(y_test, predictions_best_rf)
r2_best_gb = r2_score(y_test, predictions_best_gb)
r2_best_xgb = r2_score(y_test, predictions_best_xgb)

r2_dt = r2_score(y_test, predictions_dt)
r2_rf = r2_score(y_test, predictions_rf)
r2_gb = r2_score(y_test, predictions_gb)
r2_xgb = r2_score(y_test, predictions_xgb)


# MSE 계산
mse_best_dt = mean_squared_error(y_test, predictions_best_dt)
mse_best_rf = mean_squared_error(y_test, predictions_best_rf)
mse_best_gb = mean_squared_error(y_test, predictions_best_gb)
mse_best_xgb = mean_squared_error(y_test, predictions_best_xgb)

mse_dt = mean_squared_error(y_test, predictions_dt)
mse_rf = mean_squared_error(y_test, predictions_rf)
mse_gb = mean_squared_error(y_test, predictions_gb)
mse_xgb = mean_squared_error(y_test, predictions_xgb)

# MAE 계산
mae_best_dt = mean_absolute_error(y_test, predictions_best_dt)
mae_best_rf = mean_absolute_error(y_test, predictions_best_rf)
mae_best_gb = mean_absolute_error(y_test, predictions_best_gb)
mae_best_xgb = mean_absolute_error(y_test, predictions_best_xgb)

mae_dt = mean_absolute_error(y_test, predictions_dt)
mae_rf = mean_absolute_error(y_test, predictions_rf)
mae_gb = mean_absolute_error(y_test, predictions_gb)
mae_xgb = mean_absolute_error(y_test, predictions_xgb)

print(f"최적 Decision Tree Regressor: R² = {r2_best_dt}, MSE = {mse_best_dt}, MAE = {mae_best_dt}")
print(f"기본 Decision Tree Regressor: R² = {r2_dt}, MSE = {mse_dt}, MAE = {mae_dt}")

print(f"최적 Random Forest Regressor: R² = {r2_best_rf}, MSE = {mse_best_rf}, MAE = {mae_best_rf}")
print(f"기본 Random Forest Regressor: R² = {r2_rf}, MSE = {mse_rf}, MAE = {mae_rf}")

print(f"최적 Gradient Boosting Regressor: R² = {r2_best_gb}, MSE = {mse_best_gb}, MAE = {mae_best_gb}")
print(f"기본 Gradient Boosting Regressor: R² = {r2_gb}, MSE = {mse_gb}, MAE = {mae_gb}")

print(f"최적 XGB Regressor: R² = {r2_best_xgb}, MSE = {mse_best_xgb}, MAE = {mae_best_xgb}")
print(f"기본 XGB Regressor: R² = {r2_xgb}, MSE = {mse_xgb}, MAE = {mae_xgb}")

최적 Decision Tree Regressor: R² = 0.9189698969412907, MSE = 15.014647080848455, MAE = 2.600182148334266
기본 Decision Tree Regressor: R² = 0.9162114528909524, MSE = 15.52577889908257, MAE = 2.2747706422018346
최적 Random Forest Regressor: R² = 0.9036768332413978, MSE = 17.848408184082626, MAE = 2.5613798165137633
기본 Random Forest Regressor: R² = 0.9111223121739317, MSE = 16.468782164862425, MAE = 2.5696614678899086
최적 Gradient Boosting Regressor: R² = 0.9287101861898782, MSE = 13.209799252542433, MAE = 2.7626201627653395
기본 Gradient Boosting Regressor: R² = 0.9226691829525602, MSE = 14.329179929583848, MAE = 2.9504560675962472
최적 XGB Regressor: R² = 0.9354488505949916, MSE = 11.961143950132847, MAE = 2.5223103168032583
기본 XGB Regressor: R² = 0.9562923647133895, MSE = 8.098900208622634, MAE = 1.7272039063479923


R², MSE, MAE가 가장 적합한 최종모델(기본 XGB Regressor)을 선정한 뒤 csv파일로 저장

In [ ]:
# 예측값에 100을 곱하고 소수점 두 자리까지 반올림
adjusted_predictions = (predictions_best_xgb * 100).round(2)

# 실제 정답값에 100을 곱하고 소수점 두 자리까지 반올림
adjusted_y_test = (y_test * 100).round(2)

# 예측값과 실제 정답값을 함께 데이터프레임으로 변환
results_df = pd.DataFrame({
    '예측가격': adjusted_predictions,
    '실제가격': adjusted_y_test
})

# 결과를 CSV 파일로 저장
results_df.to_csv('predicted_vs_actual_price.csv', index=False)
print("예측값과 실제 정답값이 predicted_vs_actual_price.csv 파일로 저장되었습니다.")

예측값과 실제 정답값이 predicted_vs_actual_price.csv 파일로 저장되었습니다.
